In [1]:
# Version 2 + Bug fix - thanks to @chinhuic

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../01_toxic_proj/input/NVIDIA-apex-39e153a"))
#print(os.listdir("C:\\Users\\Bo\\Desktop\\01_toxic_proj\\input\\NVIDIA-apex-39e153a"))
#print(os.listdir("../input/NVIDIA-apex-39e153a"))
#print(os.listdir("../input/glove-global-vectors-for-word-representation"))
#print(os.listdir("../input/jigsaw-unintended-bias-in-toxicity-classification"))
#print(os.listdir("../input/fasttext-crawl-300d-2m"))

# Any results you write to the current directory are saved as output.

['.gitignore', '.nojekyll', 'apex', 'csrc', 'docs', 'examples', 'LICENSE', 'README.md', 'setup.py', 'tests']


In [2]:
import tensorflow as tf
tf.test.is_built_with_cuda()

True

In [ ]:
# Installing Nvidia Apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../01_toxic_proj/input/NVIDIA-apex-39e153a

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
#from apex import amp
import shutil

In [4]:
device=torch.device('cuda')

In [18]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
EPOCHS = 1
Data_dir="../01_toxic_proj/input/jigsaw-unintended-bias-in-toxicity-classification"
Input_dir = "../01_toxic_proj/input"
WORK_DIR = "../01_toxic_proj/working/"
num_to_load=1000000                         #Train size to match time limit
valid_size= 100000                          #Validation Size
TOXICITY_COLUMN = 'target'

In [19]:
# Add the Bart Pytorch repo to the PATH
# using files from: https://github.com/huggingface/pytorch-pretrained-BERT
package_dir_a = "../01_toxic_proj/input/pytorch-pretrained-BERT"
sys.path.insert(0, package_dir_a)

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam


In [20]:
# Translate model from tensorflow to pytorch
BERT_MODEL_PATH = '../01_toxic_proj/input/pytorch-pretrained-BERT/uncased_L-24_H-1024_A-16/'
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from C:\Users\Bo\Desktop\01_toxic_proj\input\pytorch-pretrained-BERT\uncased_L-24_H-1024_A-16\bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [1024]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [1024]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 1024]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 1024]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 1024]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_0/attenti

Loading TF weight bert/encoder/layer_13/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_13/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_13/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_13/attention/self/value/bias with shape [1024]
Loading TF weight bert/encoder/layer_13/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_13/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/layer_13/intermediate/dense/kernel with shape [1024, 4096]
Loading TF weight bert/encoder/layer_13/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_13/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_13/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_13/output/dense/kernel with shape [4096, 1024]
Loading TF weight bert/encoder/layer_14/attention/output/LayerNor

Loading TF weight bert/encoder/layer_19/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_19/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_19/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_19/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_19/attention/self/value/bias with shape [1024]
Loading TF weight bert/encoder/layer_19/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_19/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/layer_19/intermediate/dense/kernel with shape [1024, 4096]
Loading TF weight bert/encoder/layer_19/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_19/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_19/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_19/output/dense/kernel with sh

Loading TF weight bert/encoder/layer_3/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_3/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_3/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_3/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_3/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_3/attention/self/value/bias with shape [1024]
Loading TF weight bert/encoder/layer_3/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_3/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/layer_3/intermediate/dense/kernel with shape [1024, 4096]
Loading TF weight bert/encoder/layer_3/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_3/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_3/output/dense/bias wit

Loading TF weight bert/encoder/layer_9/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_9/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_9/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_9/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_9/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_9/attention/self/value/bias with shape [1024]
Loading TF weight bert/encoder/layer_9/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_9/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/layer_9/intermediate/dense/kernel with shape [1024, 4096]
Loading TF weight bert/encoder/layer_9/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_9/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_9/output/dense/bias wit

Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'output', 'LayerNorm', 'gamma']
Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'output', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'self', 'key', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'self', 'key', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'self', 'query', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'self', 'query', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'self', 'value', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_12', 'attention', 'self', 'value', 'kernel']
Initialize PyTorch 

Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'attention', 'self', 'query', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'attention', 'self', 'query', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'attention', 'self', 'value', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'attention', 'self', 'value', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'intermediate', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'intermediate', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'output', 'LayerNorm', 'gamma']
Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'output', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_17', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_18', 'attention', 'outpu

Initialize PyTorch weight ['bert', 'encoder', 'layer_21', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_21', 'output', 'LayerNorm', 'gamma']
Initialize PyTorch weight ['bert', 'encoder', 'layer_21', 'output', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_21', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'output', 'LayerNorm', 'gamma']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'output', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'self', 'key', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'self', 'key', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22'

Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'key', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'key', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'query', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'query', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'value', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'value', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'intermediate', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'intermediate', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'o

'../01_toxic_proj/working/bert_config.json'

In [22]:
os.listdir("../01_toxic_proj/working")

['bert_config.json', 'pytorch_model.bin']

In [23]:
# This is the Bert configuration file
from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig('../01_toxic_proj/input/pytorch-pretrained-BERT/uncased_L-24_H-1024_A-16/'+'bert_config.json')


In [48]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
from ipywidgets import IntProgress
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [29]:
BERT_MODEL_PATH = '../01_toxic_proj/input/pytorch-pretrained-BERT/uncased_L-24_H-1024_A-16/'


In [34]:
!pip freeze

absl-py==0.7.1
asn1crypto==0.24.0
astor==0.7.1
attrs==19.1.0
backcall==0.1.0
bleach==3.1.0
boto3==1.9.162
botocore==1.12.163
certifi==2019.3.9
cffi==1.12.3
chardet==3.0.4
colorama==0.4.1
cryptography==2.7
cycler==0.10.0
decorator==4.4.0
defusedxml==0.6.0
docutils==0.14
entrypoints==0.3
gast==0.2.2
grpcio==1.16.1
h5py==2.9.0
idna==2.8
ipykernel==5.1.1
ipython==7.5.0
ipython-genutils==0.2.0
ipywidgets==7.4.2
jedi==0.13.3
Jinja2==2.10.1
jmespath==0.9.4
joblib==0.13.2
jsonschema==3.0.1
jupyter-client==5.2.4
jupyter-core==4.4.0
Keras-Applications==1.0.8
Keras-Preprocessing==1.1.0
kiwisolver==1.1.0
Markdown==3.1.1
MarkupSafe==1.1.1
matplotlib==3.1.0
mistune==0.8.4
mkl-fft==1.0.12
mkl-random==1.0.2
mkl-service==2.0.2
mock==3.0.5
nbconvert==5.5.0
nbformat==4.4.0
nltk==3.4.1
notebook==5.7.8
numpy==1.16.4
pandas==0.24.2
pandocfilters==1.4.2
parso==0.4.0
patsy==0.5.1
pickleshare==0.7.5
Pillow==6.0.0
prometheus-client==0.6.0
prompt-toolkit==2.0.9
protobuf==3.8.0
pycparser==2.19
Pygments==2.4.2
pyO

G:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\IPython\utils\_process_win32.py:131: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
G:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\IPython\utils\_process_win32.py:131: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
G:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\IPython\utils\_process_win32.py:131: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


In [35]:
!conda install -c conda-forge ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base -c defaults conda


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [41]:
%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv")).sample(num_to_load+valid_size,random_state=SEED)
print('loaded %d records' % len(train_df))

loaded 1100000 records
Wall time: 17.5 s


In [45]:
train_df.head

<bound method NDFrame.head of               id    target                                       comment_text  \
458232    806064  0.000000  It's difficult for many old people to keep up ...   
272766    576402  0.166667  She recognized that her tiny-handed husband is...   
339129    658508  0.000000  HPHY76,\nGood for you for thinking out loud, w...   
773565   5066714  0.500000  And I bet that in the day you expected your Je...   
476233    828147  0.000000  Kennedy will add a much needed and scientifica...   
317668    631532  0.300000  Yeah, because it is far more important to pres...   
839167   5147509  0.000000          Housing inmates is a profitable business.   
913935   5237995  0.000000                                           Go Sens!   
904703   5226678  0.200000  Lol...sounds like someone IS brainwashed but I...   
1579488  6054559  0.000000  Tax relief should include commercial real esta...   
963680   5297004  0.100000  It's so simple to trash Trump the job should b...  

In [42]:
# Make sure all comment_text values are strings
train_df['comment_text'] = train_df['comment_text'].astype(str) 

In [46]:
from ipywidgets import IntProgress

In [49]:
sequences = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)

ImportError: IntProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
train_df=train_df.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']

In [40]:
%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv")).sample(num_to_load+valid_size,random_state=SEED)
print('loaded %d records' % len(train_df))

# Make sure all comment_text values are strings
train_df['comment_text'] = train_df['comment_text'].astype(str) 

sequences = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_df=train_df.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']

train_df = train_df.drop(['comment_text'],axis=1)
# convert target to 0,1
train_df['target']=(train_df['target']>=0.5).astype(float)

loaded 1100000 records


ImportError: IntProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [38]:

X = sequences[:num_to_load]                
y = train_df[y_columns].values[:num_to_load]
X_val = sequences[num_to_load:]                
y_val = train_df[y_columns].values[num_to_load:]


NameError: name 'sequences' is not defined

In [ ]:
test_df=train_df.tail(valid_size).copy()
train_df=train_df.head(num_to_load)

In [ ]:


train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))


In [ ]:
output_model_file = "bert_pytorch.bin"

lr=2e-5
batch_size = 32
accumulation_steps=2
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

model = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels=len(y_columns))
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()   # Bug fix - thanks to @chinhuic
    for i,(x_batch, y_batch) in tk0:
#        optimizer.zero_grad()
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)


torch.save(model.state_dict(), output_model_file)


In [ ]:
# Run validation
# The following 2 lines are not needed but show how to download the model for prediction
model = BertForSequenceClassification(bert_config,num_labels=len(y_columns))
model.load_state_dict(torch.load(output_model_file ))
model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()




In [ ]:
# From baseline kernel

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)


In [ ]:

MODEL_NAME = 'model1'
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))